In [ ]:
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import io
from google.colab import files
from sklearn.model_selection import train_test_split
import warnings; warnings.simplefilter('ignore')

In [ ]:
def import_dataset():
  for _ in range(4):
    uploaded = files.upload()
  income_data = pd.read_csv('income_data.csv')
  user_data = pd.read_csv('user_data.csv')
  task_data = pd.read_csv('task_data.csv')
  assignment_data = pd.read_csv('assignment_data.csv')
  return income_data, user_data, task_data, assignment_data

In [ ]:
income_data, user_data, task_data, assignment_data = import_dataset()

Saving task_data.csv to task_data.csv


Saving income_data.csv to income_data.csv


Saving user_data.csv to user_data.csv


Saving assignment_data.csv to assignment_data.csv


In [ ]:
uploaded = files.upload()
wh = pd.read_csv("cleaned_data (2).csv")

Saving cleaned_data (2).csv to cleaned_data (2).csv


In [ ]:
user_data.uid.unique().size

21

In [ ]:
task_data.columns

Index(['Unnamed: 0', 'project_id', 'pool_id', 'title', 'description',
       'pool_startedAt', 'hasInstructions', 'mayContainAdultContent',
       'requesterID', 'requesterTrusted', 'lang', 'grade',
       'averageAcceptanceTimeSec', 'moneyAvgHourly', 'moneyMed', 'moneyTop10',
       'moneyMax3', 'reward'],
      dtype='object')

In [ ]:
assignment_data.columns

Index(['Unnamed: 0', 'uid', 'project_id'], dtype='object')

In [ ]:
work_history = wh[["user_id", "title"]]

In [ ]:
user_id = user_data.uid.unique().tolist()
job_id = task_data.project_id.unique().tolist()
work_history = work_history[work_history.user_id.isin(user_id)]
work_history.dropna(axis=0,subset=["title"], inplace=True)

application_record = assignment_data[(assignment_data.uid.isin(user_id))&(assignment_data.project_id.isin(job_id))]
application_record.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1053 entries, 0 to 1052
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  1053 non-null   int64
 1   uid         1053 non-null   int64
 2   project_id  1053 non-null   int64
dtypes: int64(3)
memory usage: 32.9 KB


filter out users who are not in  application_record

In [ ]:
work_user_id = work_history.user_id.unique()
application_user_id = application_record.uid.unique()
user_set = user_data[(user_data.uid.isin(work_user_id))&(user_data.uid.isin(application_user_id))]
user_id = user_set.uid.unique()
application_record = application_record[application_record.uid.isin(user_id)]
work_history = work_history[work_history.user_id.isin(user_id)]

In [ ]:
application_record.info(), work_history.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 376 entries, 253 to 1012
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  376 non-null    int64
 1   uid         376 non-null    int64
 2   project_id  376 non-null    int64
dtypes: int64(3)
memory usage: 11.8 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 385 entries, 0 to 384
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  385 non-null    float64
 1   title    385 non-null    object 
dtypes: float64(1), object(1)
memory usage: 9.0+ KB


(None, None)

In [ ]:
(user_set.uid.unique() == application_record.uid.unique()).all()

True

In [ ]:
user_set.to_csv("user_set.csv", index=False)
application_record.to_csv("application_record.csv", index=False)
task_data.to_csv("job_set.csv", index=False)
work_history.to_csv("work_history.csv", index=False)

## Negative Sampling

In [ ]:
dataset_sample = pd.DataFrame(columns = ["UserID","JobID","label"])
job_id = task_data.project_id.unique().tolist()
groups = application_record.groupby("uid")
user_ids = []
job_ids = []
labels = []

for idx, group in tqdm(groups):
    size = len(group)
    exist_job = group.project_id.unique().tolist()
    candidate_job = [i for i in job_id if i not in exist_job ]
    sample_job = np.random.randint(0,len(candidate_job),size)
    user_ids.extend([idx] * 2 * size)
    exist_job.extend([candidate_job[i] for i in sample_job])
    job_ids.extend(exist_job)
    label = [1] * size
    label.extend([0] * size)
    labels.extend(label)

dataset_sample.UserID = user_ids
dataset_sample.JobID = job_ids
dataset_sample.label = labels

100%|██████████| 4/4 [00:00<00:00, 1075.81it/s]


In [ ]:
dataset_sample.label.value_counts()

1    376
0    376
Name: label, dtype: int64

In [ ]:
dataset_sample.to_csv("dataset.csv",index=False)

## Filter out labels in HTML

In [ ]:
job_set = pd.read_csv("job_set.csv")

In [ ]:
import re

def filter_tags(htmlstr):

    re_cdata=re.compile('//<!\[CDATA\[[^>]*//\]\]>',re.I) 
    re_script=re.compile('<\s*script[^>]*>[^<]*<\s*/\s*script\s*>',re.I)#Script
    re_style=re.compile('<\s*style[^>]*>[^<]*<\s*/\s*style\s*>',re.I)#style
    re_br=re.compile('<br\s*?/?>')
    re_h=re.compile('</?\w+[^>]*>')
    re_comment=re.compile('<!--[^>]*-->')
    s=re_cdata.sub('',htmlstr)
    s=re_script.sub('',s) 
    s=re_style.sub('',s)
    s=re_br.sub('\n',s)
    s=re_h.sub('',s) 
    s=re_comment.sub('',s)
    blank_line=re.compile('\n+')
    s=blank_line.sub('\n',s)
    s=s.replace('\\r'," ")
    s=s.replace('\\t'," ")
    s=s.replace('\n'," ")
    s=s.replace('\\n'," ")
    s = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', s, flags=re.MULTILINE)
    s = re.sub(r'[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?', '', s, flags=re.MULTILINE)
    s = re.sub(r'(www)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', s, flags=re.MULTILINE)
    s = re.sub(r'[0-9a-zA-Z.]+@[0-9a-zA-Z.]', " ", s, flags=re.MULTILINE)
    s = re.sub('\xa0', " ", s, flags=re.MULTILINE)
    s=replaceCharEntity(s)
    return s


def replaceCharEntity(htmlstr):
    CHAR_ENTITIES={'nbsp':' ','160':' ',
        'lt':'<','60':'<',
        'gt':'>','62':'>',
        'amp':'&','38':'&',
        'quot':'"','34':'"',}
   
    re_charEntity=re.compile(r'&#?(?P<name>\w+);')
    sz=re_charEntity.search(htmlstr)
    while sz:
        entity=sz.group()
        key=sz.group('name')
        try:
            htmlstr=re_charEntity.sub(CHAR_ENTITIES[key],htmlstr,1)
            sz=re_charEntity.search(htmlstr)
        except KeyError:
            htmlstr=re_charEntity.sub('',htmlstr,1)
            sz=re_charEntity.search(htmlstr)
    return htmlstr
def repalce(s,re_exp,repl_string):
    return re_exp.sub(repl_string,s)

In [ ]:
job_set = job_set.dropna(subset=['description'])

In [ ]:
job_set.shape

(561, 18)

In [ ]:
Description = [filter_tags(i) for i in job_set.description.values]

In [ ]:
job_set.columns

Index(['Unnamed: 0', 'project_id', 'pool_id', 'title', 'description',
       'pool_startedAt', 'hasInstructions', 'mayContainAdultContent',
       'requesterID', 'requesterTrusted', 'lang', 'grade',
       'averageAcceptanceTimeSec', 'moneyAvgHourly', 'moneyMed', 'moneyTop10',
       'moneyMax3', 'reward'],
      dtype='object')

In [ ]:
job_set.drop(columns=["description"])
job_set["Description"] = Description

In [ ]:
job_set.to_csv("job_set_cleaned.csv", index=False)